# Method Magic
This tutorial illustrates some of the "magic" that occurs to implement methods. The magic goes deeper than can be elucidated in this tutorial. For a fuller description (but probably less easy to understand), see the python documentation under [the standard type hierarchy](https://docs.python.org/3/reference/datamodel.html#the-standard-type-hierarchy). Search for "instance methods".

In python, a method is simply a function that is the value of an attribute on a class object. So, for example, if `A` is a class object, and `bar` is an attribute of that class object that points to a function, then `A.bar` is a method. Typically, we achieve such a state by defining the function `bar` within the class's definition, as illustrated below.

The python interpreter invokes some magic under the hood when an instance of `A` refers to the attribute `bar`. If `a` is an instance of `A`, then `a.bar` does not evaluate to the same object as `A.bar`, even though, by inheritance, one would think that these should be the exact same object. In the case of `a.bar`, there is magic that occurs that pre-binds `a` to be the implicit first argument of a new function-like object called a "bound method". 

# First Example
The class definitions below are used in many of the examples that follow.

In [1]:
# This is a base class. Its subclass is B
class A:
    def __init__(self, x):
        self.x = x # Both A and B get this instance variable, since B's init method calls super.
        
    # Both class A and class B define this.
    def bar(self, y):
        print("Calling A.bar() on:", self, y)
    
    # This is only defined in class A, and inherited by B.
    def baz(self, y):
        print("Calling A.baz() on:", self, y)
        self.bar(y)
    
    
class B(A):
    def __init__(self, x):
        super().__init__(x)
        
        def f (self, y):
            print("Calling function f() on:", self, y)
            
        # class B defines an extra attribute called "foo", not defined by A.
        # This attribute points to a function, not a method.
        # Methods must be defined on classes, not on instances.
        self.foo = f  
        
    # Overrides the bar() method defined in parent class A
    def bar(self, y):
         print("Calling B.bar() on:", self, y)

    

# Preliminaries
Before diving into the magic, we first engage in some basics, just to refresh our memory about how classes, methods, and inheritance work.

In [2]:
a = A(1) # Create an instance of A

In [3]:
a.bar(0)  # a's bar method is gotten from class A

Calling A.bar() on: <__main__.A object at 0x10aa2e748> 0


In [4]:
b = B(2) # Create an instance of B

In [5]:
b.bar(0)  # b's bar method is gotten from class B

Calling B.bar() on: <__main__.B object at 0x10aa2e898> 0


In [6]:
a.baz(0) # a's baz method is gotten from class A

Calling A.baz() on: <__main__.A object at 0x10aa2e748> 0
Calling A.bar() on: <__main__.A object at 0x10aa2e748> 0


In [7]:
# b inherits A's baz() method, but uses its own bar() method.
b.baz(0)

Calling A.baz() on: <__main__.B object at 0x10aa2e898> 0
Calling B.bar() on: <__main__.B object at 0x10aa2e898> 0


# The Magic Begins
Now we begin diving a bit "under the hood" and exploring how methods are represented in python.

## Methods Are Just Functions
Methods are just functions when accessed from a class object, but they metamorphose into a special kind of object called a "bound method" when accessed from an instance of a class.

In [8]:
# Methods are implemented as functions.
# There is no "magic" to accessing them from the class object
# on which they are defined. An access simply returns the function.
A.bar

<function __main__.A.bar>

In [9]:
# There IS magic when accessing a method from an instance.
# The access returns a "bound method" object, not a function.
a.bar

<bound method A.bar of <__main__.A object at 0x10aa2e748>>

In [10]:
A.baz

<function __main__.A.baz>

In [11]:
a.baz

<bound method A.baz of <__main__.A object at 0x10aa2e748>>

In [12]:
B.bar

<function __main__.B.bar>

In [13]:
b.bar

<bound method B.bar of <__main__.B object at 0x10aa2e898>>

In [14]:
B.baz

<function __main__.A.baz>

In [15]:
b.baz

<bound method A.baz of <__main__.B object at 0x10aa2e898>>

# Calling Methods Directly as Functions
Because a method really is just a function, when accessed through a class object, we can call that function in a completely normal way. We can even pass to its `self` argument an object that is not an instance of the class or its sub-classes, violating the standard OOP paradigm (and likely resulting in an error, but not necessarily).

In [16]:
# Call the bar method of class B as though it were a function.
# Here we pass the object b to it, which is an instance of B
B.bar(b, 0)

Calling B.bar() on: <__main__.B object at 0x10aa2e898> 0


In [17]:
# Call the bar method of class B as though it were a function.
# Here we pass the object a to it, which is NOT instance of B, and which
# has it own version of bar. But notice that A's bar() method is not
# called here. B's is. This would never happen in an OOP paradigm.
B.bar(a, 0)

Calling B.bar() on: <__main__.A object at 0x10aa2e748> 0


In [18]:
# Call the bar method of class B as though it were a function.
# Here we pass the integer 1 to it, which is not even an instance in the hierarchy.
# It works just fine.
B.bar(1, 0)

Calling B.bar() on: 1 0


# Calling A Bound Method
When we do `a.bar(0)`, we are actually calling the bound method `a.bar` on the argument `0`. The bound method encapsulates `a` as the first arg to `bar()` and expectes to receive the remaining parameters as arguments when it is called. That is why passing `0` to `a.bar` works: the bound method object only expects one more arg because it has already bound `a` to the first argument of `A.bar`.

In [19]:
g = a.bar
g

<bound method A.bar of <__main__.A object at 0x10aa2e748>>

In [20]:
g(0)

Calling A.bar() on: <__main__.A object at 0x10aa2e748> 0


Note that the bound-method mechanism must gen up a new bound method object every time we do a method access (and hence every time we do a method invocation), as illustrated below:

In [21]:
g1 = a.bar
g2 = a.bar
g1 is g2

False

By contrast, no magic happens when we access a method simply as a normal function via its class:

In [22]:
f1 = A.bar
f2 = A.bar
f1 is f2

True

# Bound Method Attributes
Bound methods are objects, and, as such, have some attributes, for example:

In [23]:
g = a.bar
print(g)
print("g's associated function is:", g.__func__)
print("g's bound 'self' arg is:", g.__self__)
print("g's method name is:", g.__name__)

<bound method A.bar of <__main__.A object at 0x10aa2e748>>
g's associated function is: <function A.bar at 0x10aa0fa60>
g's bound 'self' arg is: <__main__.A object at 0x10aa2e748>
g's method name is: bar


# Methods cannot be defined directly on instances
We can't define a method directly on an instance. Methods can only be defined on a class object. An instance created from a class object then inherits the methods of its class parent. For example, we define the method `bar()` on class `B`, and the instance `b` has access to `bar()` via its class parent. There is no mechanism for defining a method `bar()` directly on `b`. Note that `b` does have an attribute called `foo` that points to a function. That is fine, but `foo()` is just a function, not a method.

In [24]:
# If `foo()` were a method, this would work, but it doesn't,
# even though it seems syntactically just like a method invocation.
b.foo(0)

TypeError: f() missing 1 required positional argument: 'y'

In [25]:
# Instead, we have to pass both required arguments to foo, because,
# despite appearances this is merely a function application:
b.foo(0, 1)

Calling function f() on: 0 1


# Methods can be shadowed, just like any other attribute
Because of the way inheritance works, if an instance `bb` of class `B` assigns to an attribute on itself named `bar`, this will shadow the `bar()` method it inherits from its class parent.

In [26]:
# Initially, bb inherits bar() from its class parent.
bb = B(0)
bb.bar

<bound method B.bar of <__main__.B object at 0x10aab4080>>

In [27]:
# Now we blow it away by assigning to bb a local value for bar:
bb.bar = 3
bb.bar

3

In [28]:
# If we're careful, we can still access the bar() method on our parent. There are a couple ways:
B.bar(bb, 1)
bb.__class__.bar(bb, 2)

Calling B.bar() on: <__main__.B object at 0x10aab4080> 1
Calling B.bar() on: <__main__.B object at 0x10aab4080> 2


In [29]:
# But, needless to say, the "normal" way won't work, because we've blown away "bar" locally.
bb.bar(0)

TypeError: 'int' object is not callable

# Patching Classes
A class is an object at run time, just like any other python object. As such, it can have attributes dynamically added to it even after it has been defined.

In [30]:
# Define an empty class
class C:
    pass

# Patch it with a class variable
C.color = 'red'

# Give it a declare_color() method:
def f(self):
    print("My color is", self.color)
C.declare_color = f

In [31]:
# Create an instance of C
c = C()
c

In [32]:
# It has a declare_color method
c.declare_color

<bound method f of <__main__.C object at 0x10aab49b0>>

In [33]:
# Invoke the method.
c.declare_color()

My color is red
